# Dataset

In [1]:
import pandas as pd
import torch
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split

# Creating a dataframe
df = pd.read_csv('data/pokemons.csv')
df = df.drop(['Unnamed: 0', '#', 'Name', 'Type 2', 'Total'], axis=1)
df_numeric = df.copy()

# Creating the dictionary of classes
c = list(df['Type 1'].unique())
classes = torch.tensor([x for x in range(len(c))])

# Creating the numeric dataset
for idx, type in enumerate(c):
    df_numeric.loc[df['Type 1'] == type, 'Type 1'] = idx

# Separating the dataset
y = np.array(df_numeric['Type 1'], dtype=float)
df = df.drop('Type 1', axis=1)
df = (df - df.min()) / (df.max() - df.min())
matrix = df.to_numpy()
X = torch.from_numpy(matrix).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

# Separating in train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

display(df.head(5))
display(df_numeric.head(5))


/home/matheus/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.173228,0.237838,0.179592,0.298913,0.195652,0.205128
1,0.232283,0.308108,0.236735,0.380435,0.260870,0.282051
2,0.311024,0.416216,0.318367,0.489130,0.347826,0.384615
3,0.311024,0.513514,0.481633,0.608696,0.434783,0.384615
4,0.149606,0.254054,0.155102,0.271739,0.130435,0.307692


,Type 1,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0,45,49,49,65,65,45
1,0,60,62,63,80,80,60
2,0,80,82,83,100,100,80
3,0,80,100,123,122,120,80
4,1,39,52,43,60,50,65


In [16]:
X_train[0]

tensor([0.3307, 0.5946, 0.3061, 0.5163, 0.2609, 0.2308])

# Instaciating the model

In [18]:
# Model
class PokemonClassifier(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.sequence = nn.Sequential(
            nn.Linear(6, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 18),
            nn.Softmax(dim=0)
        )
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return self.sequence(X)

# Instaciating the model
model = PokemonClassifier()
display(model)

PokemonClassifier(
  (sequence): Sequential(
    (0): Linear(in_features=6, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=18, bias=True)
    (5): Softmax(dim=0)
  )
)

# Training the model

In [41]:
# Loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)

# Write a training and evaluation loop for the model
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Training
epochs = 10
model.train()
for epoch in range(epochs):
    # 1. Forward
    pred = model(X_train)
    pred = torch.argmax(pred, 1).type(torch.float).requires_grad_(True)
    '''print(pred[:50])
    print(y_train[:50])'''

    # 2. Loss
    loss = loss_fn(pred, y_train)

    # 3. Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch: {:<5d}   |  Loss: {:.5f}'.format(epoch, loss))
    




    

Epoch: 0       |  Loss: 88077.26562
Epoch: 1       |  Loss: 88077.26562
Epoch: 2       |  Loss: 88077.26562
Epoch: 3       |  Loss: 88077.26562
Epoch: 4       |  Loss: 88077.26562
Epoch: 5       |  Loss: 88077.26562
Epoch: 6       |  Loss: 88077.26562
Epoch: 7       |  Loss: 88077.26562
Epoch: 8       |  Loss: 88077.26562
Epoch: 9       |  Loss: 88077.26562


In [ ]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
print(input)
print(output)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])
tensor([[0.3683, 0.2992, 0.3325],
        [0.5215, 0.1348, 0.3438]])


In [ ]:
'''for i in range(len(X_train)):

    # 1. Forward
    pred = model(X_train[i])

    print(pred)
    print(y_train[i])
    
    # 2. Loss
    loss = loss_fn(pred, y_train[i])


    break'''

'for i in range(len(X_train)):\n\n    # 1. Forward\n    pred = model(X_train[i])\n\n    print(pred)\n    print(y_train[i])\n    \n    # 2. Loss\n    loss = loss_fn(pred, y_train[i])\n\n\n    break'

In [ ]:
X_train[0]

tensor([0.3307, 0.5946, 0.3061, 0.5163, 0.2609, 0.2308])